# Meyer-Penny Game
Rules of the game: <br>
<ul>
    <li>Place a penny heads up in a box</li>
    <li>Q gets to flip the penny</li>
    <li>Then Picard gets to flip the penny</li>
    <li>Finally, Q gets to flip the penny again</li> 
    <li>If the penny is heads, Q wins, otherwise Picard wins</li>
</ul>
The game seems fair at first look; however, Q can employ a quantum strategy that always gives him the win. <br>
Let heads be represented by $|1\rangle$ and tails by $|0\rangle$. The coin starts in state $|1\rangle$. Instead of flipping the coin classically (using a Pauli X gate), Q applies the Hadamard gate, and the coin put into a superposition. <br>

$$H|1\rangle = \frac{|0\rangle - |1\rangle}{\sqrt{2}} = |-\rangle$$

Picard can then either choose to flip or not flip the coin resulting in one of the following two states

$$X|-\rangle = \frac{|1\rangle - |0\rangle}{\sqrt{2}} \quad I|-\rangle = \frac{|0\rangle - |1\rangle}{\sqrt{2}}$$

Since the global phase is ignored, applying another Hadamard gate during Q's turn will return the coin back to the $|1\rangle$ state, always guaranteeing Q the win.

In [1]:
import numpy as np
from qiskit import *
from qiskit import Aer
%matplotlib inline

In [3]:
q = QuantumCircuit(2)

In [4]:
q.h(0)
q.x(1)
q.h(1)
q.cx(0,1)
q.h(1)

In [5]:
q.draw()

In [6]:
meas = QuantumCircuit(2, 2)
meas.barrier(range(2))
# map the quantum measurement to the classical bits
meas.measure(range(2),range(2))
qc = q+meas
qc.draw()

In [7]:
backend_sim = Aer.get_backend('qasm_simulator')
job_sim = execute(qc, backend_sim, shots=1024)
result_sim = job_sim.result()

In [8]:
counts = result_sim.get_counts(qc)
print(counts)

{'10': 516, '11': 508}


For completeness, let's step through the circuit to see how the math works out. The circuit starts out with both qubits initialized to the zero state. 

$$|\Psi_0\rangle = |0\rangle \otimes |0\rangle = \begin{bmatrix} 1 \\ 0 \end{bmatrix} \otimes \begin{bmatrix} 1 \\ 0 \end{bmatrix} = \begin{bmatrix} 1\\0\\0\\0 \end{bmatrix} \quad |00\rangle$$

Applying the Hadamard matrix to the top input and the Pauli X to the bottom input gives the state below. In this state, qubit one will always be in state $|1\rangle$, but qubit zero has a 50/50 probability of being in either state. The Hadamard is to represent Picard's choice to either flip or not flip the coin.

$$|\Psi_1\rangle = H \otimes X |\Psi_0\rangle = \frac{1}{\sqrt{2}}\begin{bmatrix} 0&1&0&1 \\ 1&0&1&0 \\ 0&1&0&-1 \\ 1&0&-1&0 \end{bmatrix} \begin{bmatrix} 1\\0\\0\\0 \end{bmatrix} = \frac{1}{\sqrt{2}} \begin{bmatrix} 0\\1\\0\\1 \end{bmatrix} \quad \frac{|01\rangle + |11\rangle}{\sqrt{2}}$$

In the next step we apply the Hadamard matrix to qubit one to represent Q's sneaky trick to flip the coin non-traditionally.

$$|\Psi_2\rangle = I \otimes H |\Psi_1\rangle = \frac{1}{\sqrt{2}}\begin{bmatrix} 1&1&0&0 \\ 1&-1&0&1 \\ 0&0&1&1 \\ 0&0&1&-1 \end{bmatrix} \frac{1}{\sqrt{2}} \begin{bmatrix} 0\\1\\0\\1 \end{bmatrix} = \frac{1}{2} \begin{bmatrix} 1\\-1\\1\\-1 \end{bmatrix} \quad \frac{|00\rangle - |01\rangle+|10\rangle - |11\rangle}{2}$$

We then apply a CNOT gate with qubit zero as the control bit and qubit one as the target bit. Depending on Picard's choice whether or not to flip the coin (using the Hadamard gate), qubit one will be flipped or not.

$$|\Psi_3\rangle = CNOT(0,1)|\Psi_2\rangle = \begin{bmatrix} 1&0&0&0 \\ 0&1&0&0 \\ 0&0&0&1 \\ 0&0&1&0 \end{bmatrix} \frac{1}{2} \begin{bmatrix} 1\\-1\\1\\-1 \end{bmatrix} = \frac{1}{2} \begin{bmatrix} 1\\-1\\-1\\1 \end{bmatrix} \quad \frac{|00\rangle - |01\rangle-|10\rangle + |11\rangle}{2}$$

Applying a final Hadamard gate as Q's last move brings us back to $|\Psi_1\rangle$. Again, the global phase is ignored, and qubit one will always be in state $|1\rangle$. Qubit zero still has a 50/50 probability of being in either state. Therefore Q always wins.

$$|\Psi_4\rangle = I \otimes H |\Psi_3\rangle = \frac{1}{\sqrt{2}}\begin{bmatrix} 1&1&0&0 \\ 1&-1&0&1 \\ 0&0&1&1 \\ 0&0&1&-1 \end{bmatrix} \frac{1}{2} \begin{bmatrix} 1\\-1\\-1\\1 \end{bmatrix} = \frac{1}{\sqrt{2}} \begin{bmatrix} 0\\1\\0\\-1 \end{bmatrix} \quad \frac{|01\rangle - |11\rangle}{\sqrt{2}}$$